## Pandas `pivot` and `pivot_table`

## Overview

Pandas provides powerful methods for reshaping data. Two of these are:

- **`pivot`**: Rearranges a DataFrame from a long format to a wide format. It requires that the combination of the pivot index and columns is unique (i.e., no duplicate entries).  
- **`pivot_table`**: Similar to pivot but more flexible. It allows you to aggregate duplicate values using an aggregation function, making it ideal when the data has multiple observations for a given index/column combination.

Both tools are useful when you want to create a summary table or convert your 'tidy' data back into a wide format for reporting or visualization.

## The Pivot Method

### What is Pivot?

The **`pivot`** function reshapes the DataFrame by specifying:
- **`index`**: The column(s) to set as the new row labels.
- **`columns`**: The column(s) whose unique values will become the new column headers.
- **`values`**: The column(s) that hold the data values for populating the new table.

### Example

Imagine you have student scores in long format for different subjects. Each student appears multiple times, once per subject:

In [1]:
import pandas as pd

In [2]:
# Sample DataFrame in long format
df = pd.DataFrame({
    'Student': ['Alice', 'Bob', 'Charlie', 'Alice', 'Bob', 'Charlie', 'Alice', 'Bob', 'Charlie'],
    'Subject': ['Maths', 'Maths', 'Maths', 'English', 'English', 'English', 'Science', 'Science', 'Science'],
    'Score': [90, 80, 70, 85, 75, 95, 88,  82, 89]
})

In [3]:
df

,Student,Subject,Score
0,Alice,Maths,90
1,Bob,Maths,80
2,Charlie,Maths,70
3,Alice,English,85
4,Bob,English,75
5,Charlie,English,95
6,Alice,Science,88
7,Bob,Science,82
8,Charlie,Science,89


In [4]:
# Pivot the DataFrame to create a wide format table
df_pivot = df.pivot(
    index='Student',
    columns='Subject',
    values='Score')

In [5]:
df_pivot

Subject,English,Maths,Science
Student,,,
Alice,85,90,88
Bob,75,80,82
Charlie,95,70,89


In this example, the pivot function rearranges the data so that each student has one row, and each subject becomes a separate column with the corresponding scores.

## The `pivot_table` Method

### What is `pivot_table`?

The `pivot_table` function is similar to pivot but designed to deal with duplicate entries in the data. When there are multiple rows for a combination of index and column values, pivot_table allows you to apply an aggregation function, such as `mean`, `median`, `sum`, and `count` (default is `mean`), to compute a single value.

### Pivot_table Example

Consider a scenario where monthly sales records are recorded multiple times for the same region and month, perhaps because of different transactions that need to be aggregated:

In [6]:
# Sample sales data with duplicate entries for a region and month
df_sales = pd.DataFrame({
    'Region': ['North', 'North', 'South', 'South', 'East', 'East', 'West', 'West'],
    'Month': ['Jan', 'Jan', 'Jan', 'Jan', 'Feb', 'Feb', 'Mar', 'Mar'],
    'Sales': [200, 220, 150, 160, 310, 305, 250, 260],
    'Rep': ['Alice', 'Bob', 'Charlie', 'David',  'Eve', 'Frank', 'Grace', 'Harriet']
})

In [7]:
df_sales

,Region,Month,Sales,Rep
0,North,Jan,200,Alice
1,North,Jan,220,Bob
2,South,Jan,150,Charlie
3,South,Jan,160,David
4,East,Feb,310,Eve
5,East,Feb,305,Frank
6,West,Mar,250,Grace
7,West,Mar,260,Harriet


In [8]:
# Use pivot_table to aggregate sales for duplicate Region/Month combinations
sales_pivot_table = df_sales.pivot_table(
    index='Region',
    columns='Month',
    values='Sales',
    aggfunc='sum'
)

In [9]:
sales_pivot_table

Month,Feb,Jan,Mar
Region,,,
East,615.0,NaN,NaN
North,NaN,420.0,NaN
South,NaN,310.0,NaN
West,NaN,NaN,510.0


When there's no data available for a given Region and Month, the table displays `NaN`.

### Pivot_table Example: Multiple Aggregations

Sometimes you may need more than one summary statistic. For example, if you want to compute both the total and average sales per region for each month:

In [10]:
df_sales

,Region,Month,Sales,Rep
0,North,Jan,200,Alice
1,North,Jan,220,Bob
2,South,Jan,150,Charlie
3,South,Jan,160,David
4,East,Feb,310,Eve
5,East,Feb,305,Frank
6,West,Mar,250,Grace
7,West,Mar,260,Harriet


In [11]:
# Compute both sum and mean for Sales using pivot_table
sales_summary = df_sales.pivot_table(
    index='Region',
    columns='Month',
    values='Sales',
    aggfunc={'Sales': ['sum', 'mean']},
    fill_value=0  # Replace missing values with 0 for clarity
)

In [12]:
sales_summary

mean                sum          
Month     Feb    Jan    Mar  Feb  Jan  Mar
Region                                    
East    307.5    0.0    0.0  615    0    0
North     0.0  210.0    0.0    0  420    0
South     0.0  155.0    0.0    0  310    0
West      0.0    0.0  255.0    0    0  510

The resulting DataFrame will have a multi-level column index with one level for the aggregation type and one for the month. This structured summary lets you quickly assess both the total and average sales for each region over different months.

## Real-World Example Applications

### Sales Data Analysis

Imagine you manage a retail business and have detailed monthly sales data for each region in a long format. You can use **pivot** or **pivot_table** to create a summary where:
- Rows represent each region.
- Columns represent months.
- Cells contain the corresponding sales figures.

#### Using Pivot (if each region-month combination is unique):

In [13]:
# Sample sales data with unique values
df_unique_sales = pd.DataFrame({
    'Region': ['North', 'South', 'East', 'West'],
    'Jan_Sales': [200, 150, 300, 250],
    'Feb_Sales': [220, 160, 310, 260],
    'Mar_Sales': [210, 155, 305, 255]
})

In [14]:
df_unique_sales

,Region,Jan_Sales,Feb_Sales,Mar_Sales
0,North,200,220,210
1,South,150,160,155
2,East,300,310,305
3,West,250,260,255


Convert the data from wide to long format for demonstration purposes. This example does it manually, but see melt_function_tutorial.ipynb.

In [15]:
df_long = pd.DataFrame({
    'Region': ['North', 'South', 'East', 'West'] * 3,
    'Month': ['Jan', 'Jan', 'Jan', 'Jan', 'Feb', 'Feb', 'Feb', 'Feb', 'Mar', 'Mar', 'Mar', 'Mar'],
    'Sales': [200, 150, 300, 250, 220, 160, 310, 260, 210, 155, 305, 255]
})

In [16]:
df_long

,Region,Month,Sales
0,North,Jan,200
1,South,Jan,150
2,East,Jan,300
3,West,Jan,250
4,North,Feb,220
5,South,Feb,160
6,East,Feb,310
7,West,Feb,260
8,North,Mar,210
9,South,Mar,155


Pivot the long format data back to wide format using pivot.

In [17]:
sales_pivot = df_long.pivot(index='Region', columns='Month', values='Sales')

In [18]:
sales_pivot

Month,Feb,Jan,Mar
Region,,,
East,310,300,305
North,220,200,210
South,160,150,155
West,260,250,255


#### Using Pivot_table (if duplicate Region/Month entries exist):
Suppose multiple sales records exist for some Region/Month combinations.

In [19]:
df_duplicate_sales = pd.DataFrame({
    'Region': ['North', 'North', 'South', 'South', 'East', 'East', 'West', 'West', 'North', 'South'],
    'Month': ['Jan', 'Jan', 'Jan', 'Jan', 'Feb', 'Feb', 'Mar', 'Mar', 'Feb', 'Mar'],
    'Sales': [200, 220, 150, 160, 310, 305, 250, 260, 210, 155]
})

In [20]:
df_duplicate_sales

,Region,Month,Sales
0,North,Jan,200
1,North,Jan,220
2,South,Jan,150
3,South,Jan,160
4,East,Feb,310
5,East,Feb,305
6,West,Mar,250
7,West,Mar,260
8,North,Feb,210
9,South,Mar,155


Use pivot_table to summarize with the average sales.

In [21]:
sales_summary = df_duplicate_sales.pivot_table(
    index='Region',
    columns='Month',
    values='Sales',
    aggfunc='mean',
    fill_value=0
)

In [22]:
sales_summary

Month,Feb,Jan,Mar
Region,,,
East,307.5,0.0,0.0
North,210.0,210.0,0.0
South,0.0,155.0,155.0
West,0.0,0.0,255.0


This approach automatically aggregates duplicate entries, ensuring that your summary table reflects the average (or other statistical measure) for each combination of region and month.

### Example: Performance Evaluation

Imagine you have performance data from employees where each row represents a performance metric recorded for different projects and quarters:

In [23]:
df_perf = pd.DataFrame({
    'Employee': ['Alice', 'Alice', 'Bob', 'Bob', 'Charlie', 'Charlie'],
    'Quarter': ['Q1', 'Q2', 'Q1', 'Q2', 'Q1', 'Q2'],
    'Project': ['A', 'A', 'B', 'B', 'A', 'A'],
    'Rating': [4.5, 4.7, 4.2, 4.0, 4.8, 4.9]
})

In [24]:
df_perf

,Employee,Quarter,Project,Rating
0,Alice,Q1,A,4.5
1,Alice,Q2,A,4.7
2,Bob,Q1,B,4.2
3,Bob,Q2,B,4.0
4,Charlie,Q1,A,4.8
5,Charlie,Q2,A,4.9


In [25]:
# Pivot the performance data so that quarters become columns
perf_pivot = df_perf.pivot(
    index='Employee',
    columns='Quarter',
    values='Rating')

In [26]:
perf_pivot

Quarter,Q1,Q2
Employee,,
Alice,4.5,4.7
Bob,4.2,4.0
Charlie,4.8,4.9


If each employee had multiple ratings per quarter and you wish to see an overall average, then pivot_table would be the better tool:

In [27]:
# For the case with duplicate ratings per employee-quarter:
df_perf_dup = pd.DataFrame({
    'Employee': ['Alice', 'Alice', 'Bob', 'Bob', 'Charlie', 'Charlie', 'Alice', 'Bob'],
    'Quarter': ['Q1', 'Q1', 'Q1', 'Q1', 'Q2', 'Q2', 'Q2', 'Q2'],
    'Rating': [4.5, 4.6, 4.2, 4.1, 4.8, 4.9, 4.7, 4.0]
})

In [28]:
df_perf_dup

,Employee,Quarter,Rating
0,Alice,Q1,4.5
1,Alice,Q1,4.6
2,Bob,Q1,4.2
3,Bob,Q1,4.1
4,Charlie,Q2,4.8
5,Charlie,Q2,4.9
6,Alice,Q2,4.7
7,Bob,Q2,4.0


In [29]:
perf_summary = df_perf_dup.pivot_table(
    index='Employee',
    columns='Quarter',
    values='Rating',
    aggfunc='mean',
    fill_value=0
)

In [30]:
perf_summary

Quarter,Q1,Q2
Employee,,
Alice,4.55,4.70
Bob,4.15,4.00
Charlie,0.00,4.85


This aggregates the multiple ratings into a single average per employee for each quarter.

## Tips

- **Choosing Between Pivot and Pivot_table:**
  - Use **`pivot`** if you are sure that each index/column pair is unique.
  - Use **`pivot_table`** if duplicate entries exist or when you need to perform aggregation.
- **Handling Missing Data:**  
  With pivot_table, you can use the **`fill_value`** parameter to substitute missing values with a default (e.g., 0).
- **Multiple Aggregations:**  
  pivot_table allows you to pass a dictionary or list to **`aggfunc`**. This lets you compute multiple statistics (like `sum` and `mean`) simultaneously.
- **Multi-Index Columns:**  
  When using multiple aggregation functions or grouping by more than one key, the resulting DataFrame might have a MultiIndex in the columns. Use **`df.columns`** manipulation if you need a flat column structure.